![Image](actinia_logo.png)

## Introduction

Actinia is a REST service to process geographical data that can be
managed by the GRASS GIS software system. The software is designed to
expose a GRASS GIS database and many [GRASS GIS](https://grass.osgeo.org/) processing tools as a
[REST service](https://en.wikipedia.org/wiki/Representational_State_Transfer).
Hence, access to GRASS resources like raster maps,
space-time raster datasets, processing and analysis modules are
available via URL. In addition, actinia allows the cloud based processing
of data, for example all Landsat 4-8 scenes as well as all
Sentinel-2 scenes in an ephemeral database. The computational results
of ephemeral processing are available via object storage as GeoTIFF/COG
or GeoPackage files.

The actinia service consists of the *[actinia core](https://github.com/actinia-org/actinia-core)*
that provides the basic but sophisticated processing service and
*[actinia plugins](https://github.com/orgs/mundialis/repositories?q=actinia+plugins&type=all&language=&sort=)*
that provide problem specific services like NDVI computation from Sentinel-2
or Landsat data, spatio-temporal statistical analysis and many more.

The following example is a Jupyter Notebook version of the online
[actinia tutorial](https://actinia-dev.mundialis.de/tutorial/introduction.html). 

### What is REST?

The Representational state transfer ([REST](https://en.wikipedia.org/wiki/Representational_State_Transfer))
is an architectural style based on [HTTP](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol)
that uses the [request methods](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Request_methods)
GET, DELETE, POST, and PUT to manipulate and receive resources with stateless operations.

While GET requests can be send easily from a browser, POST, PUT or
DELETE request can not. To access the full potential of actinia you will
need a HTTP client, that talks all HTTP communication methods. Here we will
use this Jupyter Notebook for this purpose.

### GRASS GIS Data Management

![grass_format](images/grass_format.png)



### Persistent vs. User Database
<span style="color:red">**TODO ????**</span>


---

### actinia API documentation

* [Stable actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia.mundialis.de/api/v3/swagger.json)
* [Development actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia-dev.mundialis.de/api/v3/swagger.json)

---

### Requirements

#### Software & Modules

This tutorial assumes your are comfortable with the [Python](https://python.org) programming language. Familiarity with basic REST API concepts and usage is also assumed.

Python modules used in this tutorial are:
* [requests](http://docs.python-requests.org/)
* [json](https://docs.python.org/3/library/json.html)
* [leafmap](https://leafmap.org/)

#### Actinia API user and password

This demo requires credentials for authentication set below in **Preparation** as a variable. Another actinia instance might require different credentials.

### Helper Modules and Functions

<span style="color:red">**TODO is this needed???**</span>

Before interacting with the actinia server using Python, we will import required packages an set up a helper function to print formatted JSON using json.

***Note:*** *You may need to install two helpful browser plugins called **RESTman** and **JSON Formatter** that format JSON and makes it easier to read:*

* [RESTman extension](https://chrome.google.com/webstore/detail/restman/ihgpcfpkpmdcghlnaofdmjkoemnlijdi)
* [JSON Formatter](https://chrome.google.com/webstore/detail/json-formatter/bcjindcccaagfpapjjmafapmmgkkhgoa)

## Preparation


In [ ]:
# first, let's import the required packages.

from pprint import pprint
import sys
import json
import shutil
import time

import requests
from requests.auth import HTTPBasicAuth


To simplify our life in terms of server communication we store the credentials and REST server URL in  variables.

In [ ]:
# variables to set the actinia host, version, and user

actinia_baseurl = "https://actinia.mundialis.de"
actinia_version = "v3"
actinia_url = actinia_baseurl + "/api/" + actinia_version
actinia_auth = HTTPBasicAuth('fossgis2023', 'ieh0ahweefavicieca6g')

In [ ]:
# helper function to print formatted JSON using the json module

def print_as_json(data):
    print(json.dumps(data, indent=2))

# helper function to verify a request
def verify_request(request, success_code=200):
    if request.status_code != success_code:
        print("ERROR: actinia processing failed with status code %d!" % request.status_code)
        print("See errors below:")
        print_as_json(request.json())
        request_url = request.json()["urls"]["status"]
        requests.delete(url=request_url, auth=actinia_auth)
        raise Exception("The resource <%s> has been terminated." % request_url)

## Examples

* Data management
* Landsat and Sentinel-2 NDVI computation

### Data management

List all locations that are available in the actinia persistent database via endpoint `/locations`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Available locations:")

# print formatted JSON
print_as_json(jsonResponse)

#### List mapsets in locations

List all **mapsets** inside the location `fossgis2023_epsg25832_utm32N` via the endpoint `/locations/<location_name>/mapsets`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Mapsets in fossgis2023_epsg25832_utm32N location:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

#### List contents of a mapset

List all **raster layers** in location `fossgis2023_epsg25832_utm32N` and mapset `PERMANENT` via endpoint `/locations/<location_name>/mapsets/<mapset>/raster_layers`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/raster_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster layers in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

Request **raster information** of nDOM via endpoint `/locations/<location_name>/mapsets/<mapset>/raster_layers/<raster>`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/raster_layers/ndom"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster info of ndom in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

**Render raster** ndom via endpoint `/locations/<location_name>/mapsets/<mapset>/raster_layers/<raster>/render`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/raster_layers/ndom/render"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth, stream=True)

# # check if anything went wrong
# verify_request(request, 200)

with open('img.png', 'wb') as out_file:
    shutil.copyfileobj(request.raw, out_file)

# # get a json-encoded content of the response
# jsonResponse = request.json()

# print("Raster info of ndom in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:")

# # print formatted JSON
# print_as_json(jsonResponse["process_results"])


# import shutil

# import requests

# url = 'http://example.com/img.png'
# response = requests.get(url, stream=True)
# with open('img.png', 'wb') as out_file:
#     shutil.copyfileobj(response.raw, out_file)

Show requested png
![requested_ndom_png](img.png)

List all **vector layers** in location `fossgis2023_epsg25832_utm32N` and mapset `PERMANENT` via endpoint `/locations/<location_name>/mapsets/<mapset>/vector_layers`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/mapsets/PERMANENT/vector_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Vector layers in mapset PERMANENT of location fossgis2023_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

---

### User defined processing

Actinia provides the **process chain approach** to specify import, processing and export of geodata using the actinia GRASS GIS processing system. 


#### What is a process chain
A process chain is a list of GRASS GIS modules that will be executed in serial, based on the order of the list. GRASS GIS modules are specified as process definitions that include the name of the command, the inputs and outputs, including import and export definitions as well as the module flags.
The process chain must be formulated in JSON. 

#### Ephemeral vs Persistent Processing

The processing is always performed in an temporary ephemeral database, where the data from the persistent and user data bases are linked to.

The process can then be performed **ephemeral** or **perstistent** processing.

For the **persistent** processing the ephemeral database can be moved into the persistent user database, so that the computational results can be used in further processing steps or visualized using the actinia rendering REST calls.

For the **ephemeral** processing the ephemeral database will be removed after computation.
However, all raster and vector data that was generated during the processing can be exported using GDAL/OGR specific datatypes and stored in an object storage, outside the actinia environment. 

In both processing types we have read only access to all maps of the persistent database location that is used.

#### Creating a process chain step-by-step

Create a process chain step-by-step using the example of computing the Normalized Difference Vegetation Index (NDVI).

First, create an empty process chain

In [ ]:
process_chain = {"version": 1, "list": []}

Add first item to the process chain list.
For raster operations we need to set the **computational region** to the region of interest with the desired resolution.

***Note:*** You have to specifiy the map and mapset name: `map_name@mapset_name`

In [ ]:
# list item for g.region
region_process = {
  "id": "g_region_to_dop",
  "module": "g.region",
  "inputs": [
      {
          "param": "raster",
          "value": "dop_nir@PERMANENT"
      }
  ],
  "flags": "p"
}
process_chain["list"].append(region_process)
print_as_json(process_chain)

Then add the NDVI processing to the process chain list

In [ ]:
# list item for r.mapcalc
ndvi_process = {
  "id": "r_mapcalc_ndvi",
  "module": "r.mapcalc",
  "inputs": [
      {
          "param": "expression",
          "value": "ndvi = int(127.5 * ( float((dop_nir@PERMANENT - dop_red@PERMANENT) / (dop_nir@PERMANENT + dop_red@PERMANENT)) + 1.0 ) )"
      }
  ]
}
process_chain["list"].append(ndvi_process)
print_as_json(process_chain)

Print the statistics of the NDVI

In [ ]:
# list item for r.univar
ndvi_stats_process = {
          "id": "r_univar_ndvi",
          "module": "r.univar",
          "inputs": [
              {
                  "param": "map",
                  "value": "ndvi"
              }
          ],
          "flags": "g",
          "stdout": {"id": "ndvi_stats", "format": "kv", "delimiter": "="}
      }
process_chain["list"].append(ndvi_stats_process)
print_as_json(process_chain)

Export NDVI as COG since we calculate ephemeral:

In [ ]:
# list item for exporter
export_process = {
  "id": "exporter_ndvi",
  "module": "exporter",
  "outputs": [
    {
      "export": {
        "type": "raster",
        "format": "COG"
      },
      "param": "map",
      "value": "ndvi"
    }
  ]
}
process_chain["list"].append(export_process)
print_as_json(process_chain)

Execute the job by posting the process chain to the ephemeral endpoint `/locations/<location_name>/processing_async_export`

In [ ]:
# create a POST request to the Actinia Data API
request_url = actinia_url + "/locations/fossgis2023_epsg25832_utm32N/processing_async_export"
request = requests.post(url=request_url, auth=actinia_auth, json=process_chain)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()
print("Response with status code %d:" % request.status_code)

# print formatted JSON
print_as_json(jsonResponse)

# status url
request_url = jsonResponse["urls"]["status"]
print("status url:")
print(request_url)

Most important from the actinia response is the **status**, which should be `accepted` or `running`, and the **status url** under `urls - status`.

The processing is asynchronously which means that the request is send and you get only the status url so you can request the current status of the processing until the job is finished.

The **status** of a process can be:
* accepted: actinia received the job and will start the processing soon
* running: actinia is running the job
* finished: actinia finished the job sucessfully
* error: during running the job an error occured
* terminated: a user terminated the job

Request job until finished or error:

In [ ]:
# continue polling until finished
print(request_url)

while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)
    
# print formatted JSON
print_as_json(jsonResponse)

Visualization of maps computed with actinia in leafmap
<span style="color:red">**TODO MN Fragen, was hier falsch ist**</span>

In [ ]:
import leafmap

ndvi_url = jsonResponse["urls"]["resources"][0]


m = leafmap.Map()

# define colors as hex or RGB values
colors = [(0, 0, 0), (255, 255, 255)]
vmin = 0
vmax = 255

m.add_colorbar(colors=colors, vmin=vmin, vmax=vmax)

m.add_cog_layer(ndvi_url, name="NDVI", 
                attribution='<a href="https://grass.osgeo.org/download/data/">https://grass.osgeo.org/download/data/</a>')
# show map
m